In [2]:
import os
import shutil

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optmizer
import pandas as pd

tf.get_logger().setLevel("ERROR")

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

Num GPUs Available:  1


2023-02-01 03:34:47.550602: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-01 03:34:47.564643: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-01 03:34:47.566635: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [6]:
PROJECT = !(gcloud config get-value core/project)
PROJECT = PROJECT[0]
%env PROJECT = {PROJECT}
%env BUCKET = {PROJECT}
%env REGION = "us-central1"
SEED = 0  

env: PROJECT=qwiklabs-asl-01-0b585161b979
env: BUCKET=qwiklabs-asl-01-0b585161b979
env: REGION="us-central1"


Download data from 
https://www.kaggle.com/datasets/anshulmehtakaggl/200000-abstracts-for-seq-sentence-classification?resource=download#

In [57]:
%%bash 
gsutil cp gs://qwiklabs-asl-01-0b585161b979/pm_sentence/archive.zip /home/jupyter/pubmed_sentence/archive.zip

Copying gs://qwiklabs-asl-01-0b585161b979/pm_sentence/archive.zip...
- [1 files][240.2 MiB/240.2 MiB]                                                
Operation completed over 1 objects/240.2 MiB.                                    


In [59]:
#unzip locally
from zipfile import ZipFile

zip_path = r'/home/jupyter/pubmed_sentence/archive.zip'

folder_path = r'/home/jupyter/pubmed_sentence' 

with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(folder_path)
    

In [82]:
%%bash

gsutil cp -r /home/jupyter/pubmed_sentence/* gs://${BUCKET}/pm_sentence/


Copying file:///home/jupyter/pubmed_sentence/200k_abstracts/test.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/pubmed_sentence/200k_abstracts/train.txt [Content-Type=text/plain]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file:///home/jupyter/pubmed_sentence/200k_abstracts/dev.txt [Content-Type=text/plain]...
Copying file:///home